In [2]:
import warnings
import functools
import sys
import numpy as np
import types
import numbers

def check_nD(array, ndim, arg_name='image'):
    """
    Verify an array meets the desired ndims and array isn't empty.

    Parameters
    ----------
    array : array-like
        Input array to be validated
    ndim : int or iterable of ints
        Allowable ndim or ndims for the array.
    arg_name : str, optional
        The name of the array in the original function.

    """
    array = np.asanyarray(array)
    msg_incorrect_dim = "The parameter `%s` must be a %s-dimensional array"
    msg_empty_array = "The parameter `%s` cannot be an empty array"
    if isinstance(ndim, int):
        ndim = [ndim]
    if array.size == 0:
        raise ValueError(msg_empty_array % (arg_name))
    if not array.ndim in ndim:
        raise ValueError(msg_incorrect_dim % (arg_name, '-or-'.join([str(n) for n in ndim])))


In [4]:
two = np.zeros([2,2])
print(two)

[[0. 0.]
 [0. 0.]]


In [9]:
check_nD(two, [1,4])

ValueError: The parameter `image` must be a 1-or-4-dimensional array

In [15]:
out = np.zeros(two.shape+(1,))

In [16]:
out.shape

(2, 2, 1)

In [17]:
def _handle_input_3D(image, selem, out, mask, out_dtype=None, pixel_size=1):

    check_nD(image, [2,3]) #Need to change this
    if image.dtype not in (np.uint8, np.uint16):
        message = ('Possible precision loss converting image of type {} to '
                   'uint8 as required by rank filters. Convert manually using '
                   'skimage.util.img_as_ubyte to silence this warning.'
                   .format(image.dtype))
        warn(message, stacklevel=2)
        image = img_as_ubyte(image)

    selem = np.ascontiguousarray(img_as_ubyte(selem > 0))
    image = np.ascontiguousarray(image)

    if mask is None:
        mask = np.ones(image.shape, dtype=np.uint8)
    else:
        mask = img_as_ubyte(mask)
        mask = np.ascontiguousarray(mask)

    if image is out:
        raise NotImplementedError("Cannot perform rank operation in place.")

    if out is None:
        if out_dtype is None:
            out_dtype = image.dtype
        out = np.empty(image.shape+(pixel_size,), dtype=out_dtype)
    else:
        if len(out.shape) == 2:
            out = out.reshape(out.shape+(pixel_size,))

    is_8bit = image.dtype in (np.uint8, np.int8)

    if is_8bit:
        n_bins = 256
    else:
        # Convert to a Python int to avoid the potential overflow when we add
        # 1 to the maximum of the image.
        n_bins = int(max(3, image.max())) + 1

    if n_bins > 2**10:
        warn("Bad rank filter performance is expected due to a "
             "large number of bins ({}), equivalent to an approximate "
             "bitdepth of {:.1f}.".format(n_bins, np.log2(n_bins)),
             stacklevel=2)

    return image, selem, out, mask, n_bins